## 1. General

This section of for imports, getting an overview of all experiments, and loading the most suitable model.

In [1]:
import sys
import os
import warnings
import logging
from absl import logging as absl_logging

# ✅ Ensure Correct TensorFlow Configuration
os.environ['TFDS_DATA_DIR'] = r"/data/newc6477/VAE/Single_Beat/15_percent_Physionet/"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

# ✅ Ensure Correct Working Directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"📂 Project Root: {PROJECT_ROOT}")

# ✅ Ensure `src/` is in Python's Path
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

print(f"✅ Updated sys.path:\n{sys.path}")

# ✅ Suppress Warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
absl_logging.set_verbosity(absl_logging.ERROR)

# ✅ Import Modules
try:
    from src.utils.helper import Helper
    from src.evaluate.visualizations import Visualizations
    print("✅ Successfully imported `Helper` and `Visualizations`")
except ModuleNotFoundError as e:
    print(f"❌ Import Error: {e}")
    print("🔍 Check if `src/` has `__init__.py` and is in `sys.path`.")

# ✅ Import Other Required Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
from neurokit2.signal import signal_smooth

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns



📂 Project Root: /users/newc6477/VAE/12_Lead_VECG
✅ Updated sys.path:
['/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python310.zip', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/lib-dynload', '', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages', '/users/newc6477/VAE/12_Lead_VECG', '/users/newc6477/VAE/12_Lead_VECG/src']


2025-04-04 16:19:35.005122: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-04 16:19:35.005307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-04 16:19:35.008098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 16:19:51.924841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ Successfully imported `Helper` and `Visualizations`


In [2]:
# The resolution for saving images
DPI = 300

# The source path of the experiments and models
PATH = r"/users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/"

# Some operations take some time in computation.
# Therefore, the stored intermediate results can be used to skip the respective computation.
USE_PRECOMPUTED = True

# 2. Anomaly Detection

## 🔹 Funcs used to train and test

In [3]:
import json
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd

def evaluate_models(df_physionet_train, df_physionet_test, hyperparams_list):
    """
    Evaluate a list of models using KNN classification and return a summary table.
    
    Args:
        df_physionet_train (List[pd.DataFrame]): List of train embedding DataFrames (1 per model).
        df_physionet_test (List[pd.DataFrame]): List of test embedding DataFrames (1 per model).
        hyperparams_list (List[dict]): Corresponding list of hyperparameter dictionaries.
    
    Returns:
        pd.DataFrame: Summary table with accuracy, F1 scores, hyperparameters.
    """
    results = []

    for i in range(len(df_physionet_train)):
        print(f"\n🔍 Evaluating model {i+1}/{len(df_physionet_train)}...")

        # Extract hyperparameters and latent dimension
        hparams = hyperparams_list[i]
        latent_dim = hparams.get("latent_dimension", 8)

        # Prepare feature matrices and labels
        X_train = df_physionet_train[i].iloc[:, :latent_dim].values  
        X_test = df_physionet_test[i].iloc[:, :latent_dim].values  
        y_train = np.array(df_physionet_train[i]['diagnostic'].tolist(), dtype=int)  
        y_test = np.array(df_physionet_test[i]['diagnostic'].tolist(), dtype=int)  

        # KNN classification with hyperparameter tuning
        knn = KNeighborsClassifier()
        multi_knn = MultiOutputClassifier(knn, n_jobs=-1)

        param_grid = {"estimator__n_neighbors": [3, 5, 7, 9, 11]}
        grid = GridSearchCV(multi_knn, param_grid, scoring="accuracy", cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        # Evaluation metrics
        overall_acc = accuracy_score(y_test, y_pred)
        f1_micro = f1_score(y_test, y_pred, average="micro")
        f1_macro = f1_score(y_test, y_pred, average="macro")
        accuracy_per_label = (y_pred == y_test).mean(axis=0)

        # Print evaluation
        print("✅ Overall Accuracy:", overall_acc)
        print("✅ F1 Score (Macro):", f1_macro)
        print("✅ F1 Score (Micro):", f1_micro)
        print("✅ Accuracy per label:", accuracy_per_label)

        # Classification report
        print("✅ Classification Report:\n")
        print(classification_report(
            y_test, y_pred,
            target_names=[str(i) for i in range(y_test.shape[1])]
        ))

        # Collect results
        results.append({
            "model_index": i,
            "best_k": grid.best_params_["estimator__n_neighbors"],
            "accuracy": overall_acc,
            "f1_micro": f1_micro,
            "f1_macro": f1_macro,
            **hparams
        })

    summary_df = pd.DataFrame(results)
    return summary_df


def extract_hyperparams_from_json(json_path):
    with open(json_path, 'r') as f:
        params = json.load(f)

    # Extract key hyperparameters
    hyperparams = {
        "alpha": params["coefficients"]["alpha"],
        "beta": params["coefficients"]["beta"],
        "gamma": params["coefficients"]["gamma"],
        "latent_dimension": params["latent_dimension"],
        "learning_rate": params["learning_rate"],
        "epochs": params["epochs"]
    }

    return hyperparams

from IPython.display import display
def summarize_by_latent_dimension(results_df):
    unique_dims = results_df["latent_dimension"].unique()
    summary_tables = {}

    for dim in sorted(unique_dims):
        filtered = results_df[results_df["latent_dimension"] == dim]
        sorted_table = filtered.sort_values(by="f1_macro", ascending=False)
        summary_tables[dim] = sorted_table

        print(f"\n📏 Latent Dimension: {dim}")
        display(sorted_table)

    return summary_tables


## 🔹 Lead I Evaluation

In [17]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'I')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/I/2025-04-02_22-06-06/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/I/2025-04-02_22-42-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/I/2025-04-02_23-19-22/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/I/2025-04-02_23-59-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/I/2025-04-03_00-40-18/model_best.keras
Loaded 5 models successfully.


In [18]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}

In [19]:
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='I')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='I')

#df_physionet_train = df_physionet_train[0]
#df_physionet_test = df_physionet_test[0]
#print(type(df_physionet_train))



📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: I
    128/Unknown - 4s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 26ms/step
    127/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 26ms/step
    129/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 26ms/step
    129/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: I
     29/Unknown - 1s 23ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 22ms/step
     30/Unknown - 1s 23ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 23ms/s

In [20]:

results_df = evaluate_models(df_physionet_train,df_physionet_test, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.2823866467364225
✅ F1 Score (Macro): 0.15892081020106788
✅ F1 Score (Micro): 0.3063386723882707
✅ Accuracy per label: [0.9219295  0.90592302 0.99370952 0.99218361 0.99165421 0.95571749
 0.96365845 0.9797272  0.92650722 0.97845042 0.99673019 0.983215
 0.98016318 0.97792103 0.72141256 0.95771051 0.99349153 0.99370952
 0.98019432 0.98013204 0.98343298 0.95727454 0.89950797 0.86092427
 0.86752616 0.9584579 ]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.29      0.08      0.12      2248
           1       0.35      0.18      0.24      2635
           2       0.09      0.03      0.05       158
           3       0.03      0.04      0.03       104
           4       0.57      0.33      0.41       291
           5       0.68      0.46      0.54      1863
           6       0.33      0.08      0.13      1077
           7       0.03      0.01      0.01       565
           8       0.13      0

In [22]:
display(results_df.sort_values(by="f1_macro", ascending=False))

# Set your lead name (e.g., "I", "II")
lead_name = "I"  # change as needed

best_index = results_df.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,5,0.295061,0.323570,0.170233,2.01,0.3,0.2,24,0.002,100
4,4,5,0.295186,0.322801,0.166223,2.01,0.3,0.2,22,0.002,100
2,2,5,0.282075,0.307305,0.162744,2.01,0.3,0.2,16,0.002,100
0,0,5,0.282387,0.306339,0.158921,2.01,0.3,0.2,20,0.002,100
3,3,5,0.283632,0.310624,0.157270,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead II Evaluation


In [23]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'II')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/II/2025-04-03_01-20-47/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/II/2025-04-03_02-02-15/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/II/2025-04-03_02-43-10/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/II/2025-04-03_03-25-52/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/II/2025-04-03_04-09-20/model_best.keras
Loaded 5 models successfully.


In [24]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: II
    128/Unknown - 4s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 3s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 26ms/step
    127/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step
    129/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step
    129/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: II
     31/Unknown - 1s 23ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 23ms/step
     30/Unknown - 1s 21ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 21ms

In [25]:
results_df_II = evaluate_models(df_physionet_train_II,df_physionet_test_II, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.25121449925261585
✅ F1 Score (Macro): 0.1595001945661229
✅ F1 Score (Micro): 0.3140290296148894
✅ Accuracy per label: [0.91398854 0.89206527 0.99355381 0.99062656 0.9883844  0.93597409
 0.95805306 0.97530518 0.9113727  0.97371699 0.99451918 0.98109741
 0.97704908 0.97480693 0.70805306 0.95151345 0.99327354 0.99314898
 0.97742277 0.97433981 0.9809417  0.94335451 0.90162556 0.85117713
 0.85108371 0.9535999 ]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.30      0.17      0.22      2248
           1       0.32      0.27      0.29      2635
           2       0.05      0.02      0.03       158
           3       0.01      0.03      0.02       104
           4       0.20      0.09      0.13       291
           5       0.37      0.15      0.21      1863
           6       0.23      0.11      0.15      1077
           7       0.01      0.01      0.01       565
           8       0.25     

In [26]:
display(results_df_II.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "II"  # change as needed

best_index = results_df_II.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.261460,0.321239,0.167193,2.01,0.3,0.2,24,0.002,100
4,4,3,0.258937,0.320611,0.162153,2.01,0.3,0.2,22,0.002,100
0,0,3,0.251214,0.314029,0.159500,2.01,0.3,0.2,20,0.002,100
3,3,3,0.247509,0.304836,0.157770,2.01,0.3,0.2,18,0.002,100
2,2,3,0.248848,0.297775,0.150144,2.01,0.3,0.2,16,0.002,100


## 🔹 Lead III Evaluation


In [27]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'III')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/III/2025-04-03_04-53-48/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/III/2025-04-03_05-32-16/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/III/2025-04-03_06-12-44/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/III/2025-04-03_06-54-13/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/III/2025-04-03_07-36-41/model_best.keras
Loaded 5 models successfully.


In [28]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='III')
df_physionet_test_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='III')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: III
    129/Unknown - 4s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 4s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 3s 25ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step
    128/Unknown - 4s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 3s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: III
     29/Unknown - 1s 23ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 23ms/step
     31/Unknown - 1s 23ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 23

In [29]:
results_df_III = evaluate_models(df_physionet_train_III,df_physionet_test_III, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.22736048829098157
✅ F1 Score (Macro): 0.1343021974716267
✅ F1 Score (Micro): 0.2674217874557073
✅ Accuracy per label: [0.91644868 0.90330717 0.99227703 0.99143622 0.98825984 0.93251744
 0.95587319 0.97711136 0.91034504 0.96932611 0.99588939 0.97960264
 0.9771425  0.97421525 0.67772172 0.94849278 0.99137394 0.99299327
 0.97788989 0.97586572 0.98315272 0.9435725  0.89050822 0.85011834
 0.84980693 0.94821251]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.23      0.08      0.12      2248
           1       0.34      0.19      0.25      2635
           2       0.05      0.03      0.04       158
           3       0.00      0.00      0.00       104
           4       0.28      0.19      0.22       291
           5       0.32      0.14      0.19      1863
           6       0.13      0.05      0.07      1077
           7       0.13      0.05      0.08       565
           8       0.25     

In [30]:
display(results_df_III.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "III"  # change as needed

best_index = results_df_III.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.224714,0.269305,0.138833,2.01,0.3,0.2,22,0.002,100
1,1,3,0.214094,0.260368,0.134332,2.01,0.3,0.2,24,0.002,100
0,0,3,0.227360,0.267422,0.134302,2.01,0.3,0.2,20,0.002,100
2,2,3,0.222502,0.260983,0.129101,2.01,0.3,0.2,16,0.002,100
3,3,3,0.214966,0.256973,0.125218,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead aVR Evaluation


In [4]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVR')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVR/2025-04-03_08-19-04/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVR/2025-04-03_09-04-46/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVR/2025-04-03_09-49-15/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVR/2025-04-03_10-23-24/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVR/2025-04-03_11-04-51/model_best.keras
Loaded 5 models successfully.


In [5]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVR')
df_physionet_test_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVR')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVR
    128/Unknown - 15s 90ms/step✅ Generator exhausted normally.
130/130 [==============================] - 15s 89ms/step
    129/Unknown - 4s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 3s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    127/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 26ms/step
    128/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVR
     29/Unknown - 1s 24ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 24ms/step
     29/Unknown - 1s 24ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 

In [6]:
results_df_aVR = evaluate_models(df_physionet_train_aVR,df_physionet_test_aVR, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.29113726955655206
✅ F1 Score (Macro): 0.16788750100698455
✅ F1 Score (Micro): 0.35054795708874303
✅ Accuracy per label: [0.91131041 0.89882287 0.99386522 0.99140508 0.98938092 0.94939586
 0.95702541 0.97331216 0.90265321 0.97365471 0.99604509 0.9826856
 0.97546089 0.97773418 0.72527404 0.94924016 0.99218361 0.99311784
 0.9769868  0.97760962 0.98196936 0.94017813 0.91180867 0.87577853
 0.85276532 0.95148231]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.29      0.18      0.22      2248
           1       0.36      0.29      0.32      2635
           2       0.17      0.06      0.09       158
           3       0.00      0.00      0.00       104
           4       0.32      0.15      0.21       291
           5       0.58      0.47      0.52      1863
           6       0.25      0.14      0.18      1077
           7       0.07      0.04      0.05       565
           8       0.12      0.08      0.10      2023


In [7]:
display(results_df_aVR.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVR"  # change as needed

best_index = results_df_aVR.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.292881,0.354956,0.175168,2.01,0.3,0.2,22,0.002,100
1,1,3,0.277498,0.350226,0.171473,2.01,0.3,0.2,24,0.002,100
0,0,3,0.291137,0.350548,0.167888,2.01,0.3,0.2,20,0.002,100
3,3,3,0.280767,0.343791,0.165162,2.01,0.3,0.2,18,0.002,100
2,2,3,0.279522,0.334942,0.159539,2.01,0.3,0.2,16,0.002,100


## 🔹 Lead aVL Evaluation


In [4]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVL')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVL/2025-04-03_11-46-14/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVL/2025-04-03_12-29-31/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVL/2025-04-03_13-11-59/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVL/2025-04-03_13-52-57/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVL/2025-04-03_14-35-26/model_best.keras
Loaded 5 models successfully.


In [5]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVL')
df_physionet_test_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVL')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVL
    127/Unknown - 5s 34ms/step✅ Generator exhausted normally.
130/130 [==============================] - 5s 34ms/step
    129/Unknown - 4s 33ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 33ms/step
    129/Unknown - 4s 29ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 29ms/step
    128/Unknown - 4s 29ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 29ms/step
    129/Unknown - 4s 30ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 30ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVL
     29/Unknown - 3s 104ms/step✅ Generator exhausted normally.
32/32 [==============================] - 3s 98ms/step
     30/Unknown - 2s 64ms/step✅ Generator exhausted normally.
32/32 [==============================] - 2s 6

In [6]:
results_df_aVL = evaluate_models(df_physionet_train_aVL,df_physionet_test_aVL, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.2699613851519681
✅ F1 Score (Macro): 0.1355511671594529
✅ F1 Score (Micro): 0.2721104430486043
✅ Accuracy per label: [0.9215558  0.9103139  0.99318012 0.9911871  0.99087569 0.95011211
 0.96216368 0.97779646 0.91333458 0.97611485 0.99701046 0.98271674
 0.97922895 0.97468236 0.70932985 0.95764823 0.99277529 0.99339811
 0.97969606 0.98000747 0.98365097 0.95714998 0.89446313 0.85703164
 0.8668722  0.95805306]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.20      0.04      0.07      2248
           1       0.34      0.10      0.16      2635
           2       0.09      0.04      0.06       158
           3       0.01      0.01      0.01       104
           4       0.50      0.37      0.42       291
           5       0.64      0.32      0.43      1863
           6       0.22      0.05      0.08      1077
           7       0.01      0.00      0.00       565
           8       0.18      0.10      0.13      2023
  

In [7]:
display(results_df_aVL.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVF"  # change as needed

best_index = results_df_aVL.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.251744,0.277251,0.146579,2.01,0.3,0.2,22,0.002,100
0,0,5,0.269961,0.272110,0.135551,2.01,0.3,0.2,20,0.002,100
3,3,5,0.263702,0.251943,0.125230,2.01,0.3,0.2,18,0.002,100
1,1,5,0.263795,0.258414,0.124760,2.01,0.3,0.2,24,0.002,100
2,2,5,0.257754,0.241117,0.115746,2.01,0.3,0.2,16,0.002,100


## aVF

In [8]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVF')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVF/2025-04-03_15-16-41/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVF/2025-04-03_15-57-48/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVF/2025-04-03_16-39-16/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVF/2025-04-03_17-21-25/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/aVF/2025-04-03_18-03-54/model_best.keras
Loaded 5 models successfully.


In [9]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVF')
df_physionet_test_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVF')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVF
    127/Unknown - 20s 155ms/step✅ Generator exhausted normally.
130/130 [==============================] - 20s 152ms/step
    127/Unknown - 20s 155ms/step✅ Generator exhausted normally.
130/130 [==============================] - 20s 152ms/step
    129/Unknown - 20s 155ms/step✅ Generator exhausted normally.
130/130 [==============================] - 20s 154ms/step
    127/Unknown - 20s 155ms/step✅ Generator exhausted normally.
130/130 [==============================] - 20s 152ms/step
    127/Unknown - 20s 153ms/step✅ Generator exhausted normally.
130/130 [==============================] - 20s 150ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVF
     29/Unknown - 1s 44ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 42ms/step
     30/Unknown - 1s 26ms/step✅ Generator exhausted normally.
32/32 [===================

In [10]:
results_df_aVF = evaluate_models(df_physionet_train_aVF,df_physionet_test_aVF, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.23773044344793223
✅ F1 Score (Macro): 0.15122989157895972
✅ F1 Score (Micro): 0.29623882912856014
✅ Accuracy per label: [0.91878426 0.89988166 0.99296213 0.99096911 0.98913179 0.92594669
 0.95696313 0.97658196 0.91286746 0.97110115 0.99588939 0.98038117
 0.97717364 0.97664425 0.69917788 0.9485862  0.99358495 0.99318012
 0.97620827 0.97785874 0.98330842 0.94214001 0.89303064 0.85581714
 0.85074116 0.95213627]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.29      0.11      0.16      2248
           1       0.34      0.24      0.28      2635
           2       0.04      0.02      0.03       158
           3       0.00      0.00      0.00       104
           4       0.29      0.13      0.18       291
           5       0.10      0.03      0.05      1863
           6       0.19      0.09      0.12      1077
           7       0.09      0.04      0.05       565
           8       0.30      0.29      0.29      2023

In [11]:
display(results_df_aVF.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "aVF"  # change as needed

best_index = results_df_aVF.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.242744,0.297790,0.159258,2.01,0.3,0.2,24,0.002,100
0,0,3,0.237730,0.296239,0.151230,2.01,0.3,0.2,20,0.002,100
4,4,3,0.233433,0.285956,0.144971,2.01,0.3,0.2,22,0.002,100
2,2,3,0.230599,0.279973,0.140309,2.01,0.3,0.2,16,0.002,100
3,3,3,0.236267,0.273070,0.137856,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead V1 Evaluation

In [12]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V1')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V1/2025-04-03_18-46-22/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V1/2025-04-03_19-29-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V1/2025-04-03_20-11-31/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V1/2025-04-03_20-53-31/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V1/2025-04-03_21-35-05/model_best.keras
Loaded 5 models successfully.


In [13]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V1')
df_physionet_test_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V1')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V1
    129/Unknown - 4s 29ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 30ms/step
    128/Unknown - 4s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 4s 30ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 30ms/step
    129/Unknown - 4s 27ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 27ms/step
    128/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V1
     29/Unknown - 1s 26ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 26ms/step
     30/Unknown - 1s 25ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 24ms

In [14]:
results_df_V1 = evaluate_models(df_physionet_train_V1,df_physionet_test_V1, hyperparams_list)


🔍 Evaluating model 1/5...
✅ Overall Accuracy: 0.25681988041853515
✅ F1 Score (Macro): 0.15920740857052643
✅ F1 Score (Micro): 0.32347307315503954
✅ Accuracy per label: [0.91439337 0.89707897 0.99311784 0.98978575 0.98919407 0.95904958
 0.95699427 0.97377927 0.9040857  0.97343672 0.99560912 0.97944694
 0.97564773 0.97222222 0.70506353 0.94815022 0.99100025 0.99296213
 0.97483807 0.97359243 0.98274788 0.94852392 0.90087818 0.87197932
 0.84308047 0.95254111]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.28      0.15      0.19      2248
           1       0.32      0.22      0.26      2635
           2       0.09      0.04      0.06       158
           3       0.00      0.00      0.00       104
           4       0.42      0.48      0.44       291
           5       0.64      0.68      0.66      1863
           6       0.24      0.13      0.17      1077
           7       0.18      0.14      0.16       565
           8       0.09   

In [15]:
display(results_df_V1.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V1"  # change as needed

best_index = results_df_V1.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.263110,0.324770,0.164308,2.01,0.3,0.2,22,0.002,100
3,3,3,0.261429,0.319463,0.162721,2.01,0.3,0.2,18,0.002,100
0,0,3,0.256820,0.323473,0.159207,2.01,0.3,0.2,20,0.002,100
1,1,3,0.259809,0.322356,0.157569,2.01,0.3,0.2,24,0.002,100
2,2,3,0.251806,0.307428,0.152497,2.01,0.3,0.2,16,0.002,100


## 🔹 Lead V2 Evaluation


In [4]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V2')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_12-52-30/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_13-22-04/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_13-50-36/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_14-20-03/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_14-48-29/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_22-16-33/model_best.keras
Loaded 6 models successfully.


In [5]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V2')
df_physionet_test_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V2')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: V2
    129/Unknown - 4s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 4s 26ms/step
    128/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step
    127/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step
    128/Unknown - 3s 24ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 25ms/step
    129/Unknown - 3s 25ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 25ms/step
    128/Unknown - 3s 26ms/step✅ Generator exhausted normally.
130/130 [==============================] - 3s 26ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V2
     31/Unknown - 1s 22ms/step✅ Generator exhausted normally.
32/32 [==============================] - 1s 22ms/step
     31/Unknown - 1s 21ms/step✅ Generator exhausted normally.
32/32 [========

In [6]:
results_df_V2 = evaluate_models(df_physionet_train_V2,df_physionet_test_V2, hyperparams_list)


🔍 Evaluating model 1/6...


✅ Overall Accuracy: 0.2526781265570503
✅ F1 Score (Macro): 0.15677521063120026
✅ F1 Score (Micro): 0.31659008081128187
✅ Accuracy per label: [0.90763577 0.88948057 0.99367838 0.9870142  0.992713   0.94500498
 0.95652715 0.97237793 0.90965994 0.97256477 0.99585825 0.97801445
 0.97555431 0.97636398 0.71456153 0.94519183 0.99047085 0.9918722
 0.97381041 0.97474464 0.97947808 0.94422646 0.91261834 0.84566517
 0.84628799 0.95120204]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.24      0.15      0.19      2248
           1       0.27      0.20      0.23      2635
           2       0.00      0.00      0.00       158
           3       0.03      0.10      0.05       104
           4       0.61      0.54      0.57       291
           5       0.54      0.38      0.44      1863
           6       0.14      0.06      0.09      1077
           7       0.08      0.05      0.06       565
           8       0.11      0.06      0.08      2023
 

In [7]:
display(results_df_V2.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V2"  # change as needed

best_index = results_df_V2.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.243616,0.317660,0.166202,2.01,0.3,0.2,22,0.002,100
1,1,3,0.252118,0.318177,0.164328,2.01,0.3,0.2,24,0.002,100
0,0,3,0.252678,0.316590,0.156775,2.01,0.3,0.2,20,0.002,100
5,5,3,0.249844,0.310115,0.155825,2.01,0.3,0.2,20,0.002,100
3,3,3,0.253021,0.313514,0.155194,2.01,0.3,0.2,18,0.002,100
2,2,3,0.252740,0.306458,0.151291,2.01,0.3,0.2,16,0.002,100


## 🔹 Lead V3 Evaluation


In [12]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V2')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_12-52-30/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_13-22-04/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_13-50-36/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_14-20-03/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_15/test_is_split1/V2/2025-04-03_14-48-29/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V3')
df_physionet_test_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V3')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


In [ ]:
results_df_V3 = evaluate_models(df_physionet_train_V3,df_physionet_test_V3, hyperparams_list)


🔍 Evaluating model 1/6...


KeyboardInterrupt: 

In [ ]:
display(results_df_V3.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V3"  # change as needed

best_index = results_df_V3.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.242349,0.296655,0.133611,6.01,0.3,0.2,20,0.002,100
2,2,3,0.226601,0.290172,0.130353,2.01,0.3,0.2,16,0.002,100
3,3,3,0.260409,0.300849,0.129976,2.01,0.3,0.2,18,0.002,100
4,4,3,0.232918,0.298577,0.126163,2.01,0.3,0.2,22,0.002,100
1,1,3,0.255783,0.296323,0.125629,2.01,0.3,0.2,24,0.002,100


## 🔹 Lead V4 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V4')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_03-34-32/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_03-47-08/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_03-59-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_04-11-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_04-24-22/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V4')
df_physionet_test_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V4')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V4
     43/Unknown - 2s 38ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 38ms/step
     43/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     42/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     43/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     41/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V4
      9/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 28ms/step
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 29ms/step
    

In [ ]:
results_df_V4 = evaluate_models(df_physionet_train_V4,df_physionet_test_V4, hyperparams_list)


🔍 Evaluating model 1/5...



🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V4.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V4"  # change as needed

best_index = results_df_V4.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.258185,0.304395,0.131029,6.01,0.3,0.2,20,0.002,100
2,2,3,0.226512,0.296367,0.130109,2.01,0.3,0.2,16,0.002,100
4,4,3,0.238790,0.295293,0.123494,2.01,0.3,0.2,22,0.002,100
3,3,5,0.258808,0.294518,0.118504,2.01,0.3,0.2,18,0.002,100
1,1,7,0.274555,0.301246,0.112992,2.01,0.3,0.2,24,0.002,100


## 🔹 Lead V5 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V5')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_04-36-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_04-49-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_05-01-44/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_05-14-06/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_05-26-31/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V5')
df_physionet_test_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V5')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V5
     42/Unknown - 2s 56ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 54ms/step
     43/Unknown - 2s 37ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 37ms/step
     43/Unknown - 2s 39ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 39ms/step
     43/Unknown - 2s 36ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 36ms/step
     41/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V5
     10/Unknown - 0s 27ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
    

In [ ]:
results_df_V5 = evaluate_models(df_physionet_train_V5,df_physionet_test_V5, hyperparams_list)


🔍 Evaluating model 1/5...

🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V5.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V5"  # change as needed

best_index = results_df_V5.sort_values(by="f1_macro", ascending=False).index[0]
# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,3,0.258363,0.314567,0.140270,2.01,0.3,0.2,16,0.002,100
1,1,3,0.273843,0.321651,0.140222,2.01,0.3,0.2,24,0.002,100
4,4,3,0.260943,0.313564,0.132063,2.01,0.3,0.2,22,0.002,100
0,0,3,0.237544,0.302725,0.130556,6.01,0.3,0.2,20,0.002,100
3,3,3,0.255427,0.310122,0.128155,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead V6 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V6')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_05-38-58/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_05-51-18/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_06-03-46/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_06-16-04/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_06-28-43/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V6')
df_physionet_test_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V6')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V6
     41/Unknown - 2s 37ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 37ms/step
     41/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 1s 31ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step
     43/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V6
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
      8/Unknown - 0s 28ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 29ms/step
    

In [ ]:
results_df_V6 = evaluate_models(df_physionet_train_V6,df_physionet_test_V6, hyperparams_list)


🔍 Evaluating model 1/5...

🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V6.sort_values(by="f1_macro", ascending=False))
# Set your lead name (e.g., "I", "II")
lead_name = "V6"  # change as needed

best_index = results_df_V6.sort_values(by="f1_macro", ascending=False).index[0]

# Get corresponding folder from the folders list
best_folder = folders[best_index]

# Define path for the shared JSON file
json_path = "best_folders.json"

# Load existing JSON (or create a new one)
if os.path.exists(json_path):
    with open(json_path, "r") as f:
        best_folders = json.load(f)
else:
    best_folders = {}

# Update or add entry
best_folders[f"lead_{lead_name}"] = best_folder

# Save the updated dictionary back to JSON
with open(json_path, "w") as f:
    json.dump(best_folders, f, indent=4)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,5,0.272420,0.317695,0.133943,2.01,0.3,0.2,22,0.002,100
3,3,5,0.271797,0.311583,0.130578,2.01,0.3,0.2,18,0.002,100
2,2,5,0.268772,0.315186,0.126885,2.01,0.3,0.2,16,0.002,100
1,1,3,0.228826,0.293171,0.122945,2.01,0.3,0.2,24,0.002,100
0,0,5,0.274377,0.314188,0.121019,6.01,0.3,0.2,20,0.002,100



📏 Latent Dimension: 16


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,5,0.268772,0.315186,0.126885,2.01,0.3,0.2,16,0.002,100



📏 Latent Dimension: 18


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,5,0.271797,0.311583,0.130578,2.01,0.3,0.2,18,0.002,100


...

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,5,0.274377,0.314188,0.121019,6.01,0.3,0.2,20,0.002,100



📏 Latent Dimension: 22


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,5,0.27242,0.317695,0.133943,2.01,0.3,0.2,22,0.002,100



📏 Latent Dimension: 24


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.228826,0.293171,0.122945,2.01,0.3,0.2,24,0.002,100


## 🔹 Special Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = '/users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II'  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-14-42/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-22-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-30-02/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-37-56/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-45-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-28-26/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-36-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-44-48/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/l

In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: II
     43/Unknown - 2s 50ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 50ms/step
     42/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 45ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 48ms/step
     43/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 46ms/step
     42/Unknown - 2s 47ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 47ms/step
     42/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 46ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 47ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 48ms/step
     43/Unknown - 2s 46ms/step✅ Generator exhauste

In [ ]:
results_df = evaluate_models(df_physionet_train,df_physionet_test, hyperparams_list)


🔍 Evaluating model 1/10...



🔍 Evaluating model 2/10...

🔍 Evaluating model 3/10...

🔍 Evaluating model 4/10...

🔍 Evaluating model 5/10...

🔍 Evaluating model 6/10...

🔍 Evaluating model 7/10...

🔍 Evaluating model 8/10...

🔍 Evaluating model 9/10...

🔍 Evaluating model 10/10...
